In [ ]:
import cv2
import numpy as np


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
import tensorflow as tf
print(tf.__version__)

2.15.0


In [ ]:
train_dir = "/content/drive/MyDrive/face mask detection/Train"
test_dir = "/content/drive/MyDrive/face mask detection/Test"
valid_dir = "/content/drive/MyDrive/face mask detection/Validation"

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [ ]:
target_size=(96,96)
batch_size = 24

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range = 40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=target_size,
    batch_size=batch_size,
    color_mode='rgb',
    shuffle=True,
    seed=42,
    class_mode='categorical')

Found 10009 images belonging to 2 classes.


In [ ]:
valid_datagen = ImageDataGenerator(rescale=1./255)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=target_size,
    batch_size=batch_size,
    color_mode='rgb',
    shuffle=False,
    class_mode='categorical'
)

Found 800 images belonging to 2 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale= 1./255)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=target_size,
    batch_size=batch_size,
    color_mode='rgb',
    shuffle=False,
    class_mode='categorical'
)

Found 992 images belonging to 2 classes.


In [ ]:
from tensorflow.keras import models, layers
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
num_classes = 2 #With mask, Without mask
input_shape = (96,96,3)

In [ ]:
model= models.Sequential()
#1st layer
model.add(layers.Conv2D(16,(3,3), activation='relu', padding='same', input_shape=input_shape))
model.add(layers.MaxPool2D((2,2)))
#2nd layer
model.add(layers.Conv2D(32,(3,3), activation='relu', padding='same'))
model.add(layers.MaxPool2D((2,2)))
#3rd layer
model.add(layers.Conv2D(64,(3,3), activation='relu', padding='same'))
model.add(layers.MaxPool2D((2,2)))
#4th layer
model.add(layers.Conv2D(96,(3,3), activation='relu', padding='same'))
model.add(layers.MaxPool2D((2,2)))
#5th layer
model.add(layers.Conv2D(128,(3,3), activation='relu', padding='same'))
model.add(layers.MaxPool2D((2,2)))

model.add(layers.Flatten())
model.add(layers.Dense(1024))

model.add(layers.Dense(64))

model.add(layers.Dense(num_classes, activation='softmax'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 96, 96, 16)        448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 48, 48, 16)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 48, 48, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 24, 24, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 12, 12, 64)        0

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
num_epochs = 10


In [ ]:
model.fit(train_generator, steps_per_epoch=STEP_SIZE_TRAIN, epochs=num_epochs, validation_data=valid_generator, validation_steps= STEP_SIZE_VALID)

Epoch 1/10
417/417 [==============================] - 2596s 6s/step - loss: 0.3077 - accuracy: 0.8746 - val_loss: 0.1373 - val_accuracy: 0.9508
Epoch 2/10
417/417 [==============================] - 184s 441ms/step - loss: 0.2373 - accuracy: 0.9095 - val_loss: 0.1696 - val_accuracy: 0.9381
Epoch 3/10
417/417 [==============================] - 181s 434ms/step - loss: 0.2059 - accuracy: 0.9236 - val_loss: 0.1270 - val_accuracy: 0.9609
Epoch 4/10
417/417 [==============================] - 187s 448ms/step - loss: 0.1889 - accuracy: 0.9288 - val_loss: 0.1392 - val_accuracy: 0.9482
Epoch 5/10
417/417 [==============================] - 185s 443ms/step - loss: 0.1874 - accuracy: 0.9309 - val_loss: 0.0904 - val_accuracy: 0.9659
Epoch 6/10
417/417 [==============================] - 186s 446ms/step - loss: 0.1744 - accuracy: 0.9381 - val_loss: 0.1195 - val_accuracy: 0.9545
Epoch 7/10
417/417 [==============================] - 180s 431ms/step - loss: 0.1710 - accuracy: 0.9355 - val_loss: 0.0720 - v

In [ ]:
model.save('fm_Part2_cnn.keras')

In [ ]:
loss, acc = model.evaluate(test_generator, steps=STEP_SIZE_TEST)
print("The accuracy of the model is {:.3f}\nThe loss in the model is {:.3f}".format(acc,loss))

41/41 [==============================] - 387s 10s/step - loss: 0.0602 - accuracy: 0.9827
The accuracy of the model is 0.983
The loss in the model is 0.060


In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
preds= model.predict(test_generator)
y_pred= np.argmax(preds, axis = 1)
y_actual = test_generator.classes
cm = confusion_matrix(y_actual, y_pred)
print(cm)

42/42 [==============================] - 9s 206ms/step
[[470  13]
 [  5 504]]


In [ ]:
labels = ['withMask', 'withoutMask']
print(classification_report(y_actual, y_pred, target_names=labels))

              precision    recall  f1-score   support

    withMask       0.99      0.97      0.98       483
 withoutMask       0.97      0.99      0.98       509

    accuracy                           0.98       992
   macro avg       0.98      0.98      0.98       992
weighted avg       0.98      0.98      0.98       992

